In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time
import gmaps

#list api keys here
from config import (api_key, gkey)

state = 'Oklahoma'
state_code = 40

In [2]:
url = 'http://api.census.gov/data/2010/sf1?get=P0010001,NAME&for=county:*&in=state:40&key=' + api_key
response = requests.get(url).json()
county_name = []
county_code = []
census_variable_code = []
counter = 0;
for county in response:
    if (counter > 0):
        county_name.append(county[1])
        county_code.append(county[3])
        census_variable_code.append(county[0])
    counter = counter + 1
    
all_counties = {"County" : county_name,
               "County Code" : county_code,
               "Census Code" : census_variable_code}

all_counties_df = pd.DataFrame(all_counties)
all_counties_df.head()

,County,County Code,Census Code
0,Adair County,001,22683
1,Alfalfa County,003,5642
2,Atoka County,005,14182
3,Beaver County,007,5636
4,Beckham County,009,22119


In [3]:
url = 'http://api.census.gov/data/2010/sf1?get=P0010001,P0040003&for=zip+code+tabulation+area:*&in=state:40'
response = requests.get(url).json()
#print (response)
census_variable_p0010001 = []
census_variable_p0040003 = []
zip_code = []
counter = 0
for data in response:
    if (counter > 0):
        census_variable_p0010001.append(data[0])
        census_variable_p0040003.append(data[1])
        zip_code.append(data[3])
    counter = counter + 1

zip_codes_for_state = {"Census Code" : census_variable_p0010001,
                      "Census Code (P0040003)" : census_variable_p0040003,
                      "Zip Code" : zip_code}
zip_codes_for_state_df = pd.DataFrame(zip_codes_for_state)
zip_codes_for_state_df.head()

,Census Code,Census Code (P0040003),Zip Code
0,146,64,67950
1,1062,45,73002
2,20617,1417,73003
3,857,29,73004
4,9541,1013,73005


In [5]:
file_one = "./Resources/UnemploymentReport.csv"

income_df = pd.read_csv(file_one)
house_income = income_df[["Name", "Median Household Income (2016)"]]
house_income = house_income.dropna(how='any')


In [6]:
cities = house_income["Name"]
g_lat = []
g_lon= []
g_name = []

for city in cities:
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(city, gkey)
    geo_data = requests.get(target_url).json()
    g_name.append(city)
    g_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    g_lon.append(geo_data["results"][0]["geometry"]["location"]["lng"])

In [7]:
g_dict = {"Name": g_name,
          "g_lat": g_lat,
          "g_lon": g_lon}

g_df = pd.DataFrame(g_dict)

In [9]:
final_table = pd.merge(house_income, g_df, on="Name", how="left")
final_table

final_table.to_csv("./Resources/final_household_income.csv", index=False, header = True)

In [12]:
gmaps.configure(api_key=gkey)

In [13]:
locations = final_table[["g_lat", "g_lon"]].astype(float)

household_income = final_table["Median Household Income (2016)"].astype(float)

In [14]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=household_income, 
                                 dissipating=False, max_intensity=100000,
                                 point_radius = 1)


heat_layer.dissipating = False
heat_layer.max_intensity = 100000
heat_layer.point_radius = .5

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [63]:
crime_data = pd.read_csv('./Resources/crime-report.csv', encoding="utf-8")
crime_data_ok = crime_data[crime_data['state_abbr'] == 'OK']
crime_data_processed = pd.DataFrame()
crime_data_processed['Agency Name'] = crime_data_ok['agency_name']
crime_data_processed['County'] = crime_data_ok['primary_county']
crime_data_processed['FIPS'] = crime_data_ok['primary_county_fips']
clean_df = crime_data_processed.dropna()
grouped_county_crime = clean_df.groupby(['County'])
#print(grouped_county_crime.count())

In [92]:
highschool_dropout_data = pd.read_csv('./Resources/EducationReport-hd.csv', encoding="utf-8")
hs_data_processed = pd.DataFrame()
#highschool_dropout_data.head()
hs_data_processed['County'] = highschool_dropout_data['Unnamed: 1']
hs_data_processed['FIPS'] = highschool_dropout_data['Unnamed: 0']
hs_data_processed['Dropout Percentage'] = highschool_dropout_data['Unnamed: 8']

clean_hs_df = hs_data_processed.dropna()
temp_df = clean_hs_df[clean_hs_df['County'] != 'Name']
temp_df2 = temp_df[temp_df['County'] != 'Oklahoma']
print(temp_df2)

                      County   FIPS Dropout Percentage
3          Adair, OK          40001              20.5%
4        Alfalfa, OK          40003              11.5%
5          Atoka, OK          40005              18.2%
6         Beaver, OK          40007              15.8%
7        Beckham, OK          40009              15.5%
8         Blaine, OK          40011              15.0%
9          Bryan, OK          40013              13.9%
10         Caddo, OK          40015              14.2%
11      Canadian, OK          40017               8.2%
12        Carter, OK          40019              13.5%
13      Cherokee, OK          40021              14.3%
14       Choctaw, OK          40023              19.4%
15      Cimarron, OK          40025              15.5%
16     Cleveland, OK          40027               8.7%
17          Coal, OK          40029              17.1%
18      Comanche, OK          40031              10.4%
19        Cotton, OK          40033              12.3%
20        